<a href="https://colab.research.google.com/github/Marcozambeli/Power-Systems-Stability-and-Control/blob/main/C%C3%A1lculo_das_Correntes_de_Curto_Circuito_do_Gerador_S%C3%ADncrono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo da Corrente de Curto-Circuito em um Gerador Síncrono

Nesse bloco de notas, objetiva-se o cálculo da corrente de curto-circuito em um gerador síncrono a partir das expressões de tensão e corrente obtidas a partir do circuito equivalente do gerador síncrono representado no sistema referencial $dq$.

Para isso, empregaremos as bibliotecas **numpy**, **control** e **scipy.signal**. Para essa aplicação, a biblioteca **numpy** será empregada para a realização operações matemáticas e vetoriais, enquanto a biblioteca **control** proporcionará o ferramental necessário para realizar operações com funções de transferência no domínio de Laplace. Por fim, da biblioca **scipy.signal** empregaremos a rotina *residue* para a obtenção de polos e resíduos das funções de transferência associadas às correntes da máquina.

Adicionalmente, empregaremos as bibliotecas **matplotlib** e **pandas** para a produção de gráficos e visualização de dados em forma de tabelas, respectivamente.

In [ ]:
!pip install control
import numpy as np
import control as ctrl
from scipy.signal import residue
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option("display.precision", 4)

Os comandos abaixo permitem o traçado de gráficos interativos que permitem mover as curvas e focar determidas regiões de interesse.

In [ ]:
# Obtenção de gráficos interativos
makeInteractivePlots = False
if makeInteractivePlots:
  !pip install mpld3
  import mpld3
  mpld3.enable_notebook()

## Entrada de dados

Os dados aqui empregados referem-se a uma máquina síncrona de 1458 [MVA], 25 [kV] e 60 [Hz], como detalhados abaixo.

In [ ]:
# Valores base da máquina
MVAb = 1458.
kVb = 25.
Ib = MVAb/kVb/(np.sqrt(3))
fb = 60.
wb = 2*np.pi*fb
Zb = kVb**2/MVAb
Lb = Zb/wb

# Indutâncias padronizadas
Ld = 1.60*Lb
Lq = 1.52*Lb
L1d = 0.4690*Lb
L1q = 0.8750*Lb
L2d = 0.3190*Lb
L2q = 0.3190*Lb
Ll = 0.27*Lb

# Constantes de tempo de circuito aberto
T1do = 6.20
T1qo = 2.00
T2do = 0.054
T2qo = 0.200

# Constantes de tempo de curto-circuito
T1d = L1d/Ld*T1do
T2d = L2d/L1d*T2do
T1q = L1q/Lq*T1qo
T2q = L2q/L1q*T2qo

# Parâmetros adicionais
rs = 0.01*Zb
vq0 = kVb

## Definição das indutânicas operacionais da máquina

De forma a simplificar o cálculo das indutâncias operacionais da máquina síncrona, definiremos primeiramente a frequência complexa $s$ de Laplace.

In [ ]:
s = ctrl.TransferFunction([1, 0],[1])

Agora, de acordo com as definições das indutâncias operacionais $L_d(s)$ e $L_q(s)$ dadas abaixo, podemos calculá-las empregando a variável complexa $s$, definida anteriormente.
\begin{align}
L_d(s) &= L_d \frac{(1+sT'_{d})(1+sT''_{d})}{(1+sT'_{d0})(1+sT''_{d0})} & 
L_q(s) &= L_q \frac{(1+sT'_{q})(1+sT''_{q})}{(1+sT'_{q0})(1+sT''_{q0})}
\end{align} 

In [ ]:
Lds = Ld*(1+T1d*s)*(1+T2d*s)/(1+T1do*s)/(1+T2do*s)
Lqs = Lq*(1+T1q*s)*(1+T2q*s)/(1+T1qo*s)/(1+T2qo*s)

In [ ]:
Lds

In [ ]:
Lqs

Uma vez definidas as indutâncias operacionais $L_d(s)$ e $L_q(s)$, define-se as funções $V_d(s)$ e $V_q(s)$, de forma que representem apropriadamente o curto-circuito trifásico franco nos terminais da máquina. Como estudado, essas funções são definidas da seguinte forma:

\begin{align}
V_q(s) &= -\frac{v_{q0}}{s} &&& V_d(s) &= 0
\end{align}

In [ ]:
Vqs = -vq0/s
Vds = 0

Finalmente, pode-se calcular as correntes $I_d(s)$ e $I_q(s)$ através da solução dos circuitos equivalentes dos eixos $d$ e $q$.

\begin{align}
					I_d &= \frac{\phantom{-}\omega_{b}L_{q}(s) V_{q}(s) - \left(r_s + s L_{q}(s)\right)V_{d}(s)}{\Delta} \\
					I_q &= \frac{- \omega_{b} L_{d}(s) V_{d}(s) - \left( r_{s} + s L_{d}(s)\right)V_{q}(s)}{\Delta} \\
					%						\Delta &= L_{d}(s) L_{q}(s) s^{2} + s\,r_s \left(L_{d}(s) + L_{q}(s)\right) + \\ & \qquad + \left(r_{s}^{2} + \omega_{b}^{2} L_{d}(s) L_{q}(s)\right)
					\Delta &= \bigl(r_s + s L_{d}(s)\bigr) \bigl(r_{s} + s L_{q}(s)\bigr)\, + \omega_{b}^{2} L_{d}(s) L_{q}(s)
\end{align}

A função *minreal* é empregada no cálculo abaixo para o cancelamento de polos e zeros nas funções de transferência resultantes.

In [ ]:
iDelta = ctrl.minreal(1/((rs + s*Lds)*(rs + s*Lqs) + wb**2*Lds*Lqs))
Ids = ctrl.minreal((wb*Lqs*Vqs - (rs + s*Lqs)*Vds)*iDelta)
Iqs = ctrl.minreal((-wb*Lds*Vds - (rs + s*Lds)*Vqs)*iDelta)

As funções de transferência para $I_d(s)$ e $I_q(s)$ são, então listadas abaixo.

In [ ]:
Ids

In [ ]:
Iqs

## Decomposição em frações parciais das correntes em Laplace

Cada função de transferência pode ser decomposta em frações parciais empregando-se a rotina *residue* da biblioteca *scipy.signal* como mostrado a seguir. A função *residue* retorna vetores de resíduos, polos e constantes de polinômio direto. Nos casos abordados aqui, as últimas constantes não são verificadas.

In [ ]:
Id_r, Id_p, Id_k = residue(Ids.num[0][0], Ids.den[0][0])
Iq_r, Iq_p, Iq_k = residue(Iqs.num[0][0], Iqs.den[0][0])

De forma a facilitar a visualização dos resultados, será montada uma tabela com o auxílio da biblioteca *pandas*.

In [ ]:
pf_Id = {'polos': Id_p, 'residuos':Id_r}
df_Id = pd.DataFrame(pf_Id, index=np.arange(1,Id_r.size+1))
df_Id

In [ ]:
pf_Iq = {'polos': Iq_p, 'residuos':Iq_r}
df_Iq = pd.DataFrame(pf_Iq, index=np.arange(1, Iq_r.size+1))
df_Iq

## Traçado de gráficos para as correntes de curto no gerador síncrono

Para visualizarmos as curvas no domínio do tempo associadas a $I_d(s)$ e $I_q(s)$ empregamos a função *impulse_response* da biblioteca **control** como indica abaixo. Essa função toma como argumentos de entrada a função de transferência desejada e um vetor com instantes de tempo para os quais a resposta ao impulso será calculada e retorna um vetor de instantes calculados e outro com a resposta ao impulso propriamente dita.

In [ ]:
dt = 1e-5
Tf = 1
T = np.arange(0, Tf, dt)
Tout, idt = ctrl.impulse_response(Ids, T)
Tout, iqt = ctrl.impulse_response(Iqs, T)

De posse das correntes $i_d(t)$ e $i_q(t)$, podemos então calcular a corrente nos terminais da máquina $i_a(t)$ através da transformação de Park, como indicado abaixo. Note que, nessa expressão foi considerado um ângulo inicial $\varphi_0$ para o ângulo da transformação de Park. Dessa forma, pode-se verificar a corrente de curto para diferentes instantes de aplicação do curto.

\begin{align}
i_a(t) & = {\displaystyle \sqrt{\frac{2}{3}}}\,\big[i_d(t)\,\cos\left(\omega_b t + \varphi_0\right) + i_q(t)\,\sin\left(\omega_b t + \varphi_0\right)\big]
\end{align}

In [ ]:
ph0 = 0 #np.pi/2
iat = np.sqrt(2/3)*(idt*np.cos(wb*T + ph0) + iqt*np.sin(wb*T + ph0))

Após a realização dos cálculos podemos finalmente traçar os gráficos para as correntes obtidas, como se segue.

In [ ]:
# Criação da figura e eixos que conterão os gráficos
fig = plt.figure(figsize=(8, 8))
ax = fig.subplots(3, 1, sharex=True)

# Traçado da corrente id(t)
ax[0].plot(Tout, idt)
ax[0].set_ylabel('$i_d(t)$ [A]')
ax[0].grid(ls=':')

# Traçado da corrente iq(t)
ax[1].plot(Tout, iqt)
ax[1].set_ylabel('$i_q(t)$ [A]')
ax[1].grid(ls=':')

# Traçado da corrente ia(t)
ax[2].plot(Tout, iat)
ax[2].set_ylabel('$i_a(t)$ [A]')
ax[2].grid(ls=':')

# Definição da legenda do eixo do tempo
ax[-1].set_xlabel('tempo [s]')

Na sequência é incluido o codigo para análise de Fourier de alguns períodos da corrente $i_a(t)$.

In [ ]:
# Análise da série de Fourier associada a ia(t)

# Período da frequência fundamental e frequência de amostragem
Tn = 1/fb                                                      
samplingFrequency = 1/dt

# Definição do número de períodos analisados a partir do fim da curva
nper = min(int(Tf/Tn), 5)

# Número de amostras por período
nsamples_per_period = int(Tn/dt)

# Construindo o sinal para análise
signal = iat[-nper*nsamples_per_period:-1]

# Cálculo da FFT
fourierTransform = np.fft.fft(signal)/len(signal)              # Normalize amplitude
fourierTransform = fourierTransform[range(int(len(signal)/2))] # Exclude sampling frequency

# Construção do vetor de frequências
tpCount     = len(signal)
values      = np.arange(int(tpCount/2))
timePeriod  = tpCount/samplingFrequency
frequencies = values/timePeriod

# Traçado das amplitudes calculadas pela FFT
fig_fft = plt.figure(figsize=(8, 6))
ax_fft = fig_fft.subplots(1, 1)
ax_fft.stem(frequencies, abs(fourierTransform), use_line_collection=True)
ax_fft.set_xlabel('Frequência [Hz]')
ax_fft.set_ylabel('Amplitude')
ax_fft.set_xlim(-10,300)
ax_fft.grid(ls=':')